In [1]:
import pandas as pd
import requests
import json
import os
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
import datetime
import calendar
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def populate(currency,start_date,end_date):
    res = requests.get("https://coinmarketcap.com/currencies/"+currency+"/historical-data/?start="+str(start_date)+"&end="+end_date)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table))
    final = df[0].to_json(orient='records')
    type(final)
    final = json.loads(final)

    data = json.dumps(final)
    df = pd.read_json(data)

    df['day'] = [d.day for d in df['Date']]
    df['month'] = [d.month for d in df['Date']]
    size_df = len(df)
    df['name'] = currency
    df = pd.concat([df.Close, df.Date], axis=1, join_axes=[df.Close.index])
#     print(df)
    return df
#     p1.to_excel("Output/"+folder_name+"/"+currency+".xls")




    

## Remember the sequence

-----------------------------------ripple------------------------------------------------
       Close       Date
* 0   1.140000 2017-12-22
* 1   1.190000 2017-12-21
* 2   0.775964 2017-12-20
* 3   0.791257 2017-12-19
* 4   0.778407 2017-12-18
* 5   0.728366 2017-12-17
* 6   0.758642 2017-12-16                                                   
* 7   0.756181 2017-12-15                                                  
* 8   0.864263 2017-12-14
* 9   0.471063 2017-12-13
* 10  0.373541 2017-12-12
* 11  0.251691 2017-12-11
* 12  0.237333 2017-12-10
* 13  0.244708 2017-12-09
* 14  0.252125 2017-12-08
* 15  0.222823 2017-12-07
* 16  0.232544 2017-12-06
* 17  0.246101 2017-12-05
* 18  0.253571 2017-12-04
* 19  0.252558 2017-12-03
* 20  0.255492 2017-12-02
* 21  0.255699 2017-12-01
* 22  0.250878 2017-11-30
* 23  0.243064 2017-11-29
* 24  0.299152 2017-11-28
* 25  0.253927 2017-11-27
* 26  0.248818 2017-11-26
* 27  0.252393 2017-11-25
* 28  0.244642 2017-11-24
* 29  0.242402 2017-11-23
* 30  0.238859 2017-11-22
* 31  0.233271 2017-11-21


`0 -> Last date`
`size-1 -> First Date`

In [2]:
import matplotlib.pyplot as plt

def create_graph(y,c,money):
    size = len(y)
    i = 0
    x = []
    while i<size:
        x.append(i)
        i +=1
    plt.figure(figsize=(17, 6))    
    plt.plot(x,y) 
    plt.xlabel('Days')
    plt.ylabel('Money')
    plt.title('Result of Investing $'+str(money)+' in '+c)
    try:
    
        plt.savefig('Graphs/'+sheetname+str(folder_number)+'/'+c+'.png')
    
    except:
        os.mkdir('Graphs/'+sheetname+str(folder_number))
        print("Made directory")
        plt.savefig('Graphs/'+sheetname+str(folder_number)+'/'+c+'.png')
    
        
    y = pd.DataFrame(y)
    try:
        y.to_csv('Data/'+sheetname+str(folder_number)+'/'+c+'.csv')
    except:
        os.mkdir('Data/'+sheetname+str(folder_number))
        print("Made directory")
        y.to_csv('Data/'+sheetname+str(folder_number)+'/'+c+'.csv')
#     plt.show()
    

In [3]:
def calculating_returns(df,c,principle):
    size = len(df)
    first = principle
    bought_at = df.Close[size -1]
    i = size -2
    returns = []
    returns_money = []
    money = (df.Close[i] -bought_at)/bought_at *100
    returns.append(money)
    principle = float(principle)*(1+(money/100)) 
    returns_money.append(principle)
    cur = df.Close[i]
    return_object = {}
    i -= 1
    while i >= 0:
        money = (df.Close[i] - cur)/cur *100
        returns.append(money)
        principle = float(principle)*(1+(money/100)) 
        returns_money.append(principle)
        cur = df.Close[i]
        i -= 1
    total = sum(returns)
    return_object['Coin_Name'] = c
    return_object['Amount_Invested']= first
    return_object['Final_Amount'] = principle
################REMEMBER TO UNCOMMENT THIS!!!!!!!!!!!!!!!!!!!!!############################################

    create_graph(returns_money,c,first)


################REMEMBER TO UNCOMMENT THIS!!!!!!!!!!!!!!!!!!!!!############################################
   
    print(returns_money)
#     return_object['graph_values'] = returns_money
    return return_object     


In [ ]:
def authenticate(sheetname):
    scope = ["https://spreadsheets.google.com/feeds"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json',scope)
    client = gspread.authorize(creds)
    sheet = client.open(sheetname).sheet1
    print("!!!!!!!!!!!!!!CONNECTED TO SHEETS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    return sheet

def get_coin_names(col_num):
    coins = sheet.col_values(col_num)
#     invested = sheet.col_values(col_num+3)
#     pp.pprint(coins)
#     print(invested)
    return coins 
# coins = sheet.get_all_records()
# coins = sheet.row_values(4)
# coins = sheet.col_values(1)
# coins = sheet.cell(5,2).value


def get_coin_list(i):
    coin_names= get_coin_names(i)
#     print(invested)
#     print(coin_names)
    coins = []
    j = 0
    for c in coin_names:
        if j == 0:
            j = 1
            continue
        if j == 1:
            j = 2
            continue
        if c == '':
            break
        else:
            coins.append(c.lower())
    coins = coins[2:]
 
#     invested = inv[2:]
#     print(inv)
    return coins

def get_investment_list(i):
    invested = get_coin_names(i+3)    
    j = 4
    inv = []
    while True:
        if invested[j] == '':
            break
        else:
            inv.append(invested[j])
        j+=1   
    return inv

In [ ]:
columns = [1,7,13,19]
print('Enter Sheetname')
sheetname = input()
sheet = authenticate(sheetname)
i = 0
j = 0
folder_number = 1
coins = []
baskets = []
print("Enter Start Date and End Date (format:yyyymmdd eg:20171121 (for 21 November 2017))")
start_date = input("StartDate:")
end_date = input("EndDate")
# principle = 1000
for i in columns:
    j = 0
    temp = get_coin_list(i)
    inv = get_investment_list(i)
    print(temp)
    coins.append(temp)
    returns_list = []
    for c in temp:
        p = inv[j]
        j+=1
        returns = {}
        print("-----------------------------------"+str(c)+"------------------------------------------------")
        print(p)
        df = populate(c,start_date,end_date)
        returns = calculating_returns(df,c,p)
        returns_list.append(returns)
    summary = pd.DataFrame(returns_list,columns=['Coin_Name','Amount_Invested','Final_Amount'])
    try:
        summary.to_csv(('Results/'+sheetname+str(folder_number)+'.csv'))
    except:
        os.mkdir('Results/'+sheetname+str(folder_number))
        print("Made directory")
        summary.to_csv('Results/'+sheetname+str(folder_number)+'/'+c+'.csv')
    basket_summary = {}
    sum_ = 0
#     basket_summary['ID'] = folder_number +1
    for f in summary.Amount_Invested:
        sum_ += float(f)
    basket_summary['Amount_Invested'] = sum_
    sum_ = 0
    for f in summary.Final_Amount:
        sum_ += float(f)    
    basket_summary['End_sum'] = sum_
    baskets.append(basket_summary)
#     basket_summary['End_Sum']= sum(summary.Final_Amount)
#     basket_summary = pd.DataFrame(basket_summary)
#     basket_summary.to_csv(('Results/BasketSummary/'+str(folder_number)+'.csv'))
    folder_number +=1   
baskets = pd.DataFrame(baskets)
os.mkdir('Results/BasketSummary/')
baskets.to_csv('Results/BasketSummary/'+sheetname+'Total.csv')

Enter Sheetname
